# Lab 9
# Data Science 311
# Phelix Tang
    In this lab we'll be entering a machine learning competition, we're given a data set and were supposed to predict the target column, this lab is very similar to the last one we did, example we'll use a couple different functions here. Since we have binary classifications, we're not going to use a regression to predict like the in the previous labs, we're going to use ROC predict. We'll still be using a scaler to train our train and validation sets. From there we'll use the standard scaler and CLF regression that we trained on those two sets to fit our test set.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# I had to do this in the kaggle ide since my laptop was broken, and it's too old to work with python3, thie following line of code just makes it so
# the browser can see the dataframes I downloaded.

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



### 1.1 Splitting the data
    In this lab i'm using kaggles online ide, we simply import the data set in here, then split them up into our x df's and our y dfs. we also drop id and target from train_x, since train_x is what we're going to be working with, id is the index, and target is what we're looking for

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")

train_x = train.drop(columns=['target','id'])
train_y = train["target"]
test

### 1.2 Standard Scaler and preprocessing
Here we split up our data set into two, the reason we don't use 3 is because our test set is on the kaggle website, and it's unavailble to us. We then use the preprocessing standard scaler to better fit our data. I tried a couple other encodings, like ordinal and the oneencoder, but i decided to go with the one you refered to in the lab manual

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split( train_x, train_y, test_size=0.10, random_state=42)
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)

### 1.3 Training model accuracy
Here we use the logisticregression, to fit our X_Scaled above, and our y train, then we check our area roc accuracy score. X_scaled is our x vectors that we convered into z scores and y train is our ground truths from the split training model. We use an ROC curve, because we have binary classifications.

In [ ]:
clf = LogisticRegression(solver="liblinear", random_state=0).fit(X_scaled, y_train)
#reg = LinearRegression().fit(X_scaled,y_train)
roc_auc_score(y_train, clf.predict_proba(X_scaled)[:, 1])

### 1.4 Validation model accuracy
Here we check our use our standard scaler we used for the training model onto our validation set, and we check the score for that. we don't change our scaler and use the original one from our training set. This is pretty much a repeat of what we did above but with our validation set.

In [ ]:

valid_scaled = scaler.transform(X_valid)
roc_auc_score(y_valid, clf.predict_proba(valid_scaled)[:, 1])

### Complete model
Now that our score is decently high, we'll rescale the entire dataset on a new standardscaler, using all of our data, from there we'll do the same thing, and use our logisticalregression to make a prediction. We will then use these two to fit our test model and make predictions off that.

In [ ]:
scaler = preprocessing.StandardScaler().fit(train_x)
X_scaled = scaler.transform(train_x)
clf = LogisticRegression(solver="liblinear", random_state=0).fit(X_scaled, train_y)

### Final
Now that we have our model fully trained, we'll just fit our test_x onto our scaler and logisticregression, that we used for our training model. From there we'll make a CSV from our test_predict, and grab the y column('id) from our test and submit it.

In [ ]:
test_x = test.drop(columns=['id'])
test_y = test['id']
#scaler = preprocessing.StandardScaler().fit(test_x)
X_scaled = scaler.transform(test_x)
#clf = LogisticRegression(solver="liblinear", random_state=0).fit(X_scaled, test_y)
test_predict = clf.predict_proba(X_scaled)[:, 1]
#roc_auc_score(test_y, clf.predict_proba(X_scaled)[:, 1])

In [ ]:
submit = pd.DataFrame()
submit['id'] = test['id']
submit['target'] = test_predict
submit.to_csv("Submit.csv", header=submit.columns, index = False)